#  Sentinel images of 2019 

**Import Scripts directly from google earth engine  data catalog using geemap**

[ Import scripts (geemap.ee_search())](https://geemap.org/notebooks/22_import_scripts/): Returns Python scripts similar to javascripts coded in gee

**Import the neccessary libraries for the task**

In [6]:
import ee 
import geemap
import os

**Create ana interactive map and centre to nakuru**

In [2]:
# Create a map centered at (lat, lon).
Map = geemap.Map(center=[-0.235101,36.052740], zoom=13)
Map

Map(center=[-0.235101, 36.05274], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

Once the interactive map is created. Use the draw tools in the interactive map to draw a rectangle for a region of interest and run the code below

**Create feature collection from the drawn geometry**

In [3]:
Map.draw_features
nak = ee.FeatureCollection(Map.draw_features)
roi = nak.geometry()

## Get Sentinel 2A image for the drawn region 

In [4]:
# This example uses the Sentinel-2 QA band to cloud mask
# the collection.  The Sentinel-2 cloud flags are less
# selective, so the collection is also pre-filtered by the
# CLOUDY_PIXEL_PERCENTAGE flag, to use only relatively
# cloud-free granule.

# Function to mask clouds using the Sentinel-2 QA band.
# function maskS2clouds(image) {
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])
# }

# Map the function over one year of data and take the median.
# Load Sentinel-2 TOA reflectance data.
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2019-01-01', '2019-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .map(maskS2clouds)

composite = collection.median().clip(roi)
ImageInt=composite.select('B12', 'B11','B8','B4', 'B3', 'B2')

# Display the results.
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'RGB')
# Display the map.
Map

Map(bottom=262899.0, center=[-0.31242215783639027, 36.11756282066232], controls=(WidgetControl(options=['posit…

You can zoom the true color image using the zoom buttons in the interactive map to further check if it is fitting the procedure

## Export the median composite image to your local directory for further analysis in Desktop GIS LIke QGIS

In [8]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'MedianComposite.tif')

In [9]:
geemap.ee_export_image(ImageInt, filename=filename, scale=90, region=roi, file_per_band=False)
#geemap.ee_export_image(ImageInt, filename=filename, scale=90, region=roi, file_per_band=True) if you want to export individual bands

Generating URL ...
Please wait ...
Data downloaded to C:\Users\HP\Downloads\MedianComposite.tif


**For visualization and further analysis . the bands that are required for color composite area ['B8', 'B4', 'B3'], while for the false color composite are ['B12', 'B11', 'B4']. We can further calculate NDVI as shown below**

In [10]:
nir=composite.select("B8")
red=composite.select("B4")
ndvi=nir.subtract(red).divide(nir.add(red)).rename("NDVI")
visparamsNdvi={min:-1,max:1,"palette":["grey","white","green"]}
Map.addLayer(ndvi, visparamsNdvi,  'NDVI Image')
Map

Map(bottom=2101156.0, center=[-0.3179152387907839, 35.95318794250489], controls=(WidgetControl(options=['posit…

The next step is to try and Work on plotting the individual bands using [Rasterio](https://rasterio.readthedocs.io/en/latest/)